![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_RoBertaForQuestionAnswering.ipynb)

## Import ONNX RoBertaForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `RoBertaForQuestionAnswering` is only available since in `Spark NLP 5.1.4` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import RoBERTa models trained/fine-tuned for question answering via `RobertaForQuestionAnswering` or `TFRobertaForQuestionAnswering`. These models are usually under `Question Answering` category and have `roberta` in their labels
- Reference: [TFRobertaForQuestionAnswering](https://huggingface.co/docs/transformers/model_doc/roberta#transformers.TFRobertaForQuestionAnswering)
- Some [example models](https://huggingface.co/models?filter=roberta&pipeline_tag=question-answering)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.52.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [1]:
!pip install -q --upgrade transformers[onnx]==4.52.3 optimum onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models.
- We'll use [deepset/roberta-base-squad2](https://huggingface.co/deepset/roberta-base-squad2) model from HuggingFace as an example as an example and export it with the `optimum-cli`.

In [19]:
MODEL_NAME = 'deepset/roberta-base-squad2'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

!optimum-cli export onnx --model {MODEL_NAME} {ONNX_MODEL}

2025-06-16 04:48:10.059060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750049290.388172    9439 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750049290.478072    9439 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Let's have a look inside these two directories and see what we are dealing with:

In [20]:
!ls -l {ONNX_MODEL}

total 489628
-rw-r--r-- 1 root root       727 Jun 16 04:48 config.json
-rw-r--r-- 1 root root    456318 Jun 16 04:48 merges.txt
-rw-r--r-- 1 root root 496545753 Jun 16 04:48 model.onnx
-rw-r--r-- 1 root root       957 Jun 16 04:48 special_tokens_map.json
-rw-r--r-- 1 root root      1302 Jun 16 04:48 tokenizer_config.json
-rw-r--r-- 1 root root   3558642 Jun 16 04:48 tokenizer.json
-rw-r--r-- 1 root root    798293 Jun 16 04:48 vocab.json


- Spark NLP needs the vocab as a plain `vocab.txt` (so we have to convert it from Hugging Face’s `vocab.json`) and expects both `vocab.txt` and `merges.txt` to be inside an assets folder to work properly.

In [21]:
import json

with open(f'{ONNX_MODEL}/vocab.json') as f, open(f'{ONNX_MODEL}/vocab.txt', 'w') as out:
    out.writelines(f"{k}\n" for k in json.load(f).keys())

!mkdir -p {ONNX_MODEL}/assets
!mv {ONNX_MODEL}/vocab.txt {ONNX_MODEL}/merges.txt {ONNX_MODEL}/assets/

In [22]:
!ls -lR {ONNX_MODEL}

onnx_models/deepset/roberta-base-squad2:
total 489184
drwxr-xr-x 2 root root      4096 Jun 16 04:48 assets
-rw-r--r-- 1 root root       727 Jun 16 04:48 config.json
-rw-r--r-- 1 root root 496545753 Jun 16 04:48 model.onnx
-rw-r--r-- 1 root root       957 Jun 16 04:48 special_tokens_map.json
-rw-r--r-- 1 root root      1302 Jun 16 04:48 tokenizer_config.json
-rw-r--r-- 1 root root   3558642 Jun 16 04:48 tokenizer.json
-rw-r--r-- 1 root root    798293 Jun 16 04:48 vocab.json

onnx_models/deepset/roberta-base-squad2/assets:
total 848
-rw-r--r-- 1 root root 456318 Jun 16 04:48 merges.txt
-rw-r--r-- 1 root root 407065 Jun 16 04:48 vocab.txt


Voila! We have our `vocab.txt` and `merges.txt` inside assets directory

## Import and Save RoBertaForQuestionAnswering in Spark NLP


- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [11]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 40.6 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [12]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `RoBertaForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [23]:
from sparknlp.annotator import RoBertaForQuestionAnswering

spanClassifier = RoBertaForQuestionAnswering.loadSavedModel(
     ONNX_MODEL,
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [24]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [25]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your RoBertaForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [26]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 484996
drwxr-xr-x 4 root root      4096 Jun 16 04:48 fields
drwxr-xr-x 2 root root      4096 Jun 16 04:48 metadata
-rw-r--r-- 1 root root 496621657 Jun 16 04:49 roberta_classification_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBertaForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [27]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import RoBertaForQuestionAnswering
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

qa_model = RoBertaForQuestionAnswering.load(f"./{ONNX_MODEL}_spark_nlp_onnx") \
    .setInputCols(["document_question", "document_context"]) \
    .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    qa_model
])

data = [
    ("What is the boiling point of water?", "Water boils at 100 degrees Celsius."),
    ("Who is the president of the USA?", "The president of the USA is Joe Biden."),
    ("What color is the sky?", "The sky is blue on a clear day."),
]
df = spark.createDataFrame(data, ["question", "context"])

result = pipeline.fit(df).transform(df)
result.select("question", "answer.result").show(truncate=False)

+-----------------------------------+---------------------+
|question                           |result               |
+-----------------------------------+---------------------+
|What is the boiling point of water?|[100 degrees Celsius]|
|Who is the president of the USA?   |[Joe Biden]          |
|What color is the sky?             |[blue]               |
+-----------------------------------+---------------------+



That's it! You can now go wild and use hundreds of `RoBertaForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
